<a href="https://colab.research.google.com/github/emadphysics/Divulging-electricity-consumption-patterns/blob/main/consumption_2_ols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

import zipfile

# Import basic modules
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import math
import seaborn as sns
from matplotlib import rcParams
from datetime import date

from pandas.tseries.holiday import AbstractHolidayCalendar
# Import regression and error metrics modules
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Import plotly modules to view time series in a more interactive way
import plotly.graph_objects as go
import pandas as pd

# Standard scaler for preprocessing
from sklearn.preprocessing import StandardScaler

# Importing time series split for cross validation 
from sklearn.model_selection import TimeSeriesSplit

import os

plt.style.use('bmh')

# special IPython command to prepare the notebook for matplotlib and other libraries
%matplotlib inline 

#sns.set_style("whitegrid")
#sns.set_context("poster")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dict_error = dict()
# creating function for plotting predicted vs actual energy values
def plot_predvstrue_reg(pred, truth, model_name=None):

    fig, ax = plt.subplots(1,1, figsize=(8,8))
    ax.scatter(truth, pred) 
    _ = plt.xlabel("Observed energy in MWH")
    _ = plt.ylabel("Predicted energy in MWH")
    _ = plt.title("Observed vs Predicted energy using model {}".format(model_name))
    _ = plt.xlim(1000, 5000)
    _ = plt.ylim(1000, 5000)
    #plotting 45 deg line to see how the prediction differs from the observed values
    x = np.linspace(*ax.get_xlim())
    _ = ax.plot(x, x)
def error_metrics(y_pred, y_truth, model_name = None, test = True):

    if isinstance(y_pred, np.ndarray):
        y_pred = y_pred
    else:
        y_pred = y_pred.to_numpy()
        
    if isinstance(y_truth, np.ndarray):
        y_truth = y_truth
    else:
        y_truth = y_truth.to_numpy()
        
    print('\nError metrics for model {}'.format(model_name))
    
    RMSE = np.sqrt(mean_squared_error(y_truth, y_pred))
    print("RMSE or Root mean squared error: %.2f" % RMSE)
    
    # Explained variance score: 1 is perfect prediction

    R2 = r2_score(y_truth, y_pred)
    print('Variance score: %.2f' % R2 )

    MAE = mean_absolute_error(y_truth, y_pred)
    print('Mean Absolute Error: %.2f' % MAE)

    MAPE = (np.mean(np.abs((y_truth - y_pred) / y_truth)) * 100)
    print('Mean Absolute Percentage Error: %.2f %%' % MAPE)
    
    # Appending the error values along with the model_name to the dict
    if test:
        train_test = 'test'
    else:
        train_test = 'train'
    
    #df = pd.DataFrame({'model': model_name, 'RMSE':RMSE, 'R2':R2, 'MAE':MAE, 'MAPE':MAPE}, index=[0])
    name_error = ['model', 'train_test', 'RMSE', 'R2', 'MAE', 'MAPE']
    value_error = [model_name, train_test, RMSE, R2, MAE, MAPE]
    list_error = list(zip(name_error, value_error))
    
    for error in list_error:
        if error[0] in dict_error:
            # append the new number to the existing array at this slot
            dict_error[error[0]].append(error[1])
        else:
            # create a new array in this slot
            dict_error[error[0]] = [error[1]]
    #return(dict_error)
def plot_timeseries(ts, title = 'og', opacity = 1):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = ts.index, y = ts.values, name = "observed",
                         line_color = 'lightslategrey', opacity = opacity))

    fig.update_layout(title_text = title,
                  xaxis_rangeslider_visible = True)
    fig.show()
def plot_ts_pred(og_ts, pred_ts, model_name=None, og_ts_opacity = 0.5, pred_ts_opacity = 0.5):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x = og_ts.index, y = np.array(og_ts.values), name = "Observed",
                         line_color = 'deepskyblue', opacity = og_ts_opacity))

    try:
        fig.add_trace(go.Scatter(x = pred_ts.index, y = pred_ts, name = model_name,
                         line_color = 'lightslategrey', opacity = pred_ts_opacity))
    except: #if predicted values are a numpy array they won't have an index
        fig.add_trace(go.Scatter(x = og_ts.index, y = pred_ts, name = model_name,
                         line_color = 'lightslategrey', opacity = pred_ts_opacity))


    #fig.add_trace(go)
    fig.update_layout(title_text = 'Observed test set vs predicted energy MWH values using {}'.format(model_name),
                  xaxis_rangeslider_visible = True)
    fig.show()
def train_test(data, test_size = 0.15, scale = False, cols_to_transform=None, include_test_scale=False):

    df = data.copy()
    test_index = int(len(df)*(np.abs(1-test_size)))

    if scale and include_test_scale:
        scaler = StandardScaler()
        df[cols_to_transform] = scaler.fit_transform(df[cols_to_transform])
        
    X_train = df.drop('load', axis = 1).iloc[:test_index]
    y_train = df.load.iloc[:test_index]
    X_test = df.drop('load', axis = 1).iloc[test_index:]
    y_test = df.load.iloc[test_index:]
    
    # StandardScaler fit only on the training set
    if scale and not include_test_scale:
        scaler = StandardScaler()
        X_train[cols_to_transform] = scaler.fit_transform(X_train[cols_to_transform])
        X_test[cols_to_transform] = scaler.transform(X_test[cols_to_transform])
    
    return X_train, X_test, y_train, y_test


In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/frame.csv',usecols=['load', 'year', 'hour', 'month', 'day', 'weekday', 'holiday',
       'non_working', 'season', 'temp'])

In [ ]:
#we didnot consider daytime variable

In [ ]:
cat_cols = ['year','hour', 'non_working','weekday']

for col in cat_cols:
    df[col] = df[col].astype('category')

In [ ]:
df_dum = pd.get_dummies(df)

In [ ]:
m = ols('load ~  C(year)+ C(day)+  C(hour) +C(weekday)+C(non_working)+ temp' , df).fit()

In [ ]:
m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   load   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     3134.
Date:                Mon, 28 Dec 2020   Prob (F-statistic):               0.00
Time:                        14:18:18   Log-Likelihood:            -4.2484e+05
No. Observations:               43824   AIC:                         8.498e+05
Df Residuals:                   43758   BIC:                         8.504e+05
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                3.905e+04    151.802    257.232      0.000    3.88e+04    3.93e+04
C(year)[T.2016]          7274.4326     59.320    122.631      0.000    7158.165    7390.700
C(year)[T.2017]          7462.6416     59.363    125.712      0.000    7346.289    7578.994
C(year)[T.2018]          7996.9171     59.385    134.663      0.000    7880.522    8113.312
C(year)[T.2019]          7647.7128     59.391    128.769      0.000    7531.306    7764.120
C(day)[T.2]               601.8914    146.512      4.108      0.000     314.725     889.058
C(day)[T.3]              1033.2973    146.490      7.054      0.000     746.174    1320.421
C(day)[T.4]              1079.6040    146.492      7.370      0.000     792.476    1366.732
C(day)[T.5]               682.0572    146.462      4.657      0.000     394.989     969.126
C(day)[T.6]              1010.8479    146.487      6.901      0.000     723.730    1297.966
C(day)[T.7]              1134.4741    146.522      7.743      0.000     847.288    1421.660
C(day)[T.8]              1072.2482    146.457      7.321      0.000     785.189    1359.308
C(day)[T.9]               851.3653    146.512      5.811      0.000     564.199    1138.531
C(day)[T.10]              585.8757    146.456      4.000      0.000     298.820     872.932
C(day)[T.11]              928.3180    146.494      6.337      0.000     641.186    1215.450
C(day)[T.12]             1112.3365    146.459      7.595      0.000     825.274    1399.399
C(day)[T.13]              877.0797    146.449      5.989      0.000     590.036    1164.123
C(day)[T.14]              374.0385    146.505      2.553      0.011      86.886     661.191
C(day)[T.15]              875.6030    146.456      5.979      0.000     588.547    1162.659
C(day)[T.16]              800.6998    146.509      5.465      0.000     513.540    1087.860
C(day)[T.17]              671.0196    146.483      4.581      0.000     383.911     958.128
C(day)[T.18]              870.4741    146.487      5.942      0.000     583.358    1157.590
C(day)[T.19]              934.6140    146.489      6.380      0.000     647.492    1221.736
C(day)[T.20]             1109.6533    146.487      7.575      0.000     822.536    1396.771
C(day)[T.21]              785.3887    146.490      5.361      0.000     498.265    1072.512
C(day)[T.22]              963.1238    146.465      6.576      0.000     676.050    1250.197
C(day)[T.23]             1008.5860    146.518      6.884      0.000     721.408    1295.764
C(day)[T.24]              874.4715    146.500      5.969      0.000     587.328    1161.615
C(day)[T.25]              353.2683    146.465      2.412      0.016      66.193     640.343
C(day)[T.26]              383.4319    146.474      2.618      0.009      96.340     670.524
C(day)[T.27]              109.8451    146.495  